In [1]:
#pip install tensorflow

In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
directory = "TImages/"

In [3]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [4]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [5]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [6]:
create_training_data()

In [7]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [8]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [11]:
## NB MODEL

In [12]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [13]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5] # You can adjust this range
    
}

In [15]:
# Create the Gaussian Naive Bayes model
nb = GaussianNB()

In [16]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(nb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [17]:
# priors
grid_search.class_prior_ = [0.5, 0.5]

In [18]:
# Fit the GridSearch object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ................................var_smoothing=1e-09; total time=   9.8s
[CV] END ................................var_smoothing=1e-09; total time=  10.1s
[CV] END ................................var_smoothing=1e-09; total time=  10.5s
[CV] END ................................var_smoothing=1e-09; total time=  10.3s
[CV] END ................................var_smoothing=1e-09; total time=  10.3s
[CV] END ................................var_smoothing=1e-08; total time=  10.2s
[CV] END ................................var_smoothing=1e-08; total time=  10.3s
[CV] END ................................var_smoothing=1e-08; total time=  10.3s
[CV] END ................................var_smoothing=1e-08; total time=  10.2s
[CV] END ................................var_smoothing=1e-08; total time=  10.1s
[CV] END ................................var_smoothing=1e-07; total time=  10.1s
[CV] END ................................var_smoo

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             verbose=2)

In [20]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred = best_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best Gaussian Naive Bayes Parameters: {'var_smoothing': 1e-05}
Accuracy: 43.76%


In [ ]:
# work progressing...........


In [21]:
grid_search.cv_results_

{'mean_fit_time': array([5.60125275, 5.60636601, 5.59870048, 5.59512506, 5.65964108]),
 'std_fit_time': array([0.1424704 , 0.06572221, 0.04737179, 0.04403882, 0.23083343]),
 'mean_score_time': array([4.69699397, 4.72637434, 4.63817544, 4.55179038, 4.55276222]),
 'std_score_time': array([0.13155325, 0.09546034, 0.11314608, 0.12998449, 0.11550831]),
 'param_var_smoothing': masked_array(data=[1e-09, 1e-08, 1e-07, 1e-06, 1e-05],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'var_smoothing': 1e-09},
  {'var_smoothing': 1e-08},
  {'var_smoothing': 1e-07},
  {'var_smoothing': 1e-06},
  {'var_smoothing': 1e-05}],
 'split0_test_score': array([0.39316239, 0.39636752, 0.39957265, 0.40491453, 0.41132479]),
 'split1_test_score': array([0.39209402, 0.39316239, 0.39529915, 0.39850427, 0.39957265]),
 'split2_test_score': array([0.38782051, 0.39102564, 0.39529915, 0.4017094 , 0.40598291]),
 'split3_test_score': array([0.4508547 

In [22]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.601253,0.142470,4.696994,0.131553,0.0,{'var_smoothing': 1e-09},0.393162,0.392094,0.387821,0.450855,0.413462,0.407479,0.023432,5
1,5.606366,0.065722,4.726374,0.095460,0.0,{'var_smoothing': 1e-08},0.396368,0.393162,0.391026,0.454060,0.413462,0.409615,0.023588,4
2,5.598700,0.047372,4.638175,0.113146,0.0,{'var_smoothing': 1e-07},0.399573,0.395299,0.395299,0.457265,0.413462,0.412179,0.023508,3
3,5.595125,0.044039,4.551790,0.129984,0.000001,{'var_smoothing': 1e-06},0.404915,0.398504,0.401709,0.467949,0.419872,0.418590,0.025742,2
4,5.659641,0.230833,4.552762,0.115508,0.00001,{'var_smoothing': 1e-05},0.411325,0.399573,0.405983,0.472222,0.418803,0.421581,0.026096,1


In [23]:
df[['param_var_smoothing', 'params', 'mean_test_score']]

,param_var_smoothing,params,mean_test_score
0,0.0,{'var_smoothing': 1e-09},0.407479
1,0.0,{'var_smoothing': 1e-08},0.409615
2,0.0,{'var_smoothing': 1e-07},0.412179
3,0.000001,{'var_smoothing': 1e-06},0.418590
4,0.00001,{'var_smoothing': 1e-05},0.421581


In [24]:
grid_search.best_score_

0.4215811965811966

In [25]:
grid_search.best_params_

{'var_smoothing': 1e-05}